#### Importations & chargement du référentiel

In [7]:
import json
import logging
from pathlib import Path

# Configurer logs pour le notebook lui-même (optionnel)
logger = logging.getLogger("evaluator")
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler("../logs/quiz_generation.log")
fh.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
logger.addHandler(fh)

# Chargement du référentiel AI-900
with open("../referentiel/referentiel_ai900.json", encoding="utf-8") as f:
    referentiel = json.load(f)

mots_cles = [m.lower() for m in referentiel["mots_cles"]]

#### Fonction d’évaluation

In [8]:
def evaluer_pertinence(question: dict, logs: str) -> dict:
    texte = question.get("question", "").lower()
    score = sum(1 for mot in mots_cles if mot in texte)
    
    pertinence = "Oui" if score >= 2 else "Non"
    commentaire = "Pertinent." if score >= 2 else "Question probablement hors sujet AI-900."

    if "source semble peu fiable" in logs.lower():
        commentaire += " ⚠️ Source douteuse détectée."
    
    logger.info(f"Question ID {question.get('id')} : score={score}, pertinence={pertinence}")
    
    return {
        "question_id": question.get("id", "N/A"),
        "pertinence": pertinence,
        "note_sur_10": min(score * 2, 10),
        "commentaire": commentaire
    }


#### Chargement des questions et logs

In [10]:
with open("../logs/quiz_generation.log", encoding="utf-8") as f:
    logs = f.read()

# Exemple de liste de questions (à remplacer par ton fichier réel)
questions = [
    {
        "id": 1,
        "question": "Qu’est-ce que le machine learning supervisé ?",
        "réponse": "A",
        "explication": "...",
        "source": "https://learn.microsoft.com/..."
    },
    {
        "id": 2,
        "question": "Quel est l’âge de Bill Gates ?",
        "réponse": "C",
        "explication": "...",
        "source": "https://wikipedia.org/..."
    }
]


#### Évaluation en boucle + visualisation

In [11]:
résultats = [evaluer_pertinence(q, logs) for q in questions]

import pandas as pd
pd.DataFrame(résultats)


,question_id,pertinence,note_sur_10,commentaire
0,1,Oui,4,Pertinent.
1,2,Non,0,Question probablement hors sujet AI-900.
